In [1]:
exp <- "Flt3l"
outdir <- "/data2/mito_lineage/Analysis/annotation/output/data/"
prefix <- "CHIP_april08_2021_Croker/MTblacklist/"
external_dat_file <- "granja_10X_CD34.rds"
#external_dat_dir <- "/data2/mito_lineage/Analysis/annotation/output/data/"
nTop = 25000

In [2]:
library(Seurat)
#library(SeuratData)
#InstallData("panc8")

library(SummarizedExperiment)

Attaching SeuratObject

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges, rowRanks,
    rowSdDif

In [3]:
library(ggplot2)
library(cowplot)
library(patchwork)


Attaching package: ‘patchwork’


The following object is masked from ‘package:cowplot’:

    align_plots




In [4]:
genedir <- outdir
outdir <- file.path(outdir, prefix)
outdir

[1] "/data2/mito_lineage/Analysis/annotation/output/data//CHIP_april08_2021_Croker/MTblacklist/"

## Import curr data and  granja data

In [5]:
curr_SE <- readRDS(file.path(outdir, paste0(exp, ".cd34.rds")))
ext_SE <- readRDS(file.path(outdir, external_dat_file))

curr_SE[['tech']] <- exp
ext_SE[['tech']] <-'granja_cd34'

C1boo <- subset(x = ext_SE, subset = Group %in% c("BM_pDC", "CLP", "CMP", "GMP", "HSC", "LMPP", "MEP", "Monocytes", "MPP"))

DefaultAssay(C1boo) <- 'peaks'
DefaultAssay(curr_SE) <- 'peaks'


scatac.list <- c(C1boo, curr_SE)


In [10]:
# colData <- fread(file.path(external_dat_dir, "GSE129785_scATAC-Hematopoiesis-CD34.cell_barcodes.txt")) %>% data.frame()
# CD34boo <- colData$Group %in% c("CD34_Progenitors_Rep1","CD34_Progenitors_Rep2")
# C1boo <-  colData$Group %in% c("BM_pDC", "CLP", "CMP", "GMP", "HSC", "LMPP", "MEP", "Monocytes", "MPP")
# SE_CD34 <- ext_SE[,CD34boo]
# SE_C1 <- ext_SE[,C1boo]

In [13]:
# data("panc8")
# scatac.list <- SplitObject(panc8, split.by = "tech")
# scatac.list <- scatac.list[c("celseq", "celseq2", "fluidigmc1", "smartseq2")]

In [ ]:
for (i in 1:length(scatac.list)) {
    scatac.list[[i]] <- NormalizeData(scatac.list[[i]], verbose = FALSE)
    scatac.list[[i]] <- FindVariableFeatures(scatac.list[[i]], nfeatures = 2000,
        verbose = TRUE)
}

In [ ]:
reference.list <- scatac.list[c(1,2)]
scatac.anchors <- FindIntegrationAnchors(object.list = reference.list, dims = 1:30, assay = c("RNA","RNA"))


In [ ]:
scatac.integrated <- IntegrateData(anchorset = scatac.anchors, dims = 1:30)

In [ ]:

# switch to integrated assay. The variable features of this assay are automatically set during
# IntegrateData
DefaultAssay(scatac.integrated) <- "integrated"
# Run the standard workflow for visualization and clustering
scatac.integrated <- ScaleData(scatac.integrated, verbose = FALSE)
scatac.integrated <- RunPCA(scatac.integrated, npcs = 30, verbose = FALSE)
scatac.integrated <- RunUMAP(scatac.integrated, reduction = "pca", dims = 1:30, verbose = FALSE)
p1 <- DimPlot(scatac.integrated, reduction = "umap", group.by = "tech")
p2 <- DimPlot(scatac.integrated, reduction = "umap", group.by = "Group", label = TRUE, repel = TRUE) +
    NoLegend()
p1 + p2

In [ ]:
scatac.query <- scatac.list[[2]]
scatac.anchors <- FindTransferAnchors(reference = scatac.integrated, query = scatac.query,
    dims = 1:30, reference.reduction = "pca")
predictions <- TransferData(anchorset = scatac.anchors, refdata = scatac.integrated$Group,
    dims = 1:30)
scatac.query <- AddMetaData(scatac.query, metadata = predictions)

In [ ]:
scatac.query$prediction.match <- scatac.query$predicted.id == scatac.query$Group
table(scatac.query$prediction.match)


In [ ]:
table(scatac.query$predicted.id)


In [ ]:
scatac.integrated <- RunUMAP(scatac.integrated, dims = 1:30, reduction = "pca", return.model = TRUE)
scatac.query <- MapQuery(anchorset = scatac.anchors, reference = scatac.integrated, query = scatac.query,
    refdata = list(celltype = "celltype"), reference.reduction = "pca", reduction.model = "umap")

In [ ]:
p1 <- DimPlot(scatac.integrated, reduction = "umap", group.by = "celltype", label = TRUE, label.size = 3,
    repel = TRUE) + NoLegend() + ggtitle("Reference annotations")
p2 <- DimPlot(scatac.query, reduction = "ref.umap", group.by = "predicted.celltype", label = TRUE,
    label.size = 3, repel = TRUE) + NoLegend() + ggtitle("Query transferred labels")
p1 + p2

# Unimodal UMAP Projection


In [ ]:
scatac.integrated <- RunUMAP(scatac.integrated, dims = 1:30, reduction = "pca", return.model = TRUE)
scatac.query <- MapQuery(anchorset = scatac.anchors, reference = scatac.integrated, query = scatac.query,
    refdata = list(celltype = "celltype"), reference.reduction = "pca", reduction.model = "umap")

In [ ]:
scatac.query <- TransferData(anchorset = scatac.anchors, reference = scatac.integrated, query = scatac.query,
    refdata = list(celltype = "celltype"))
scatac.query <- IntegrateEmbeddings(anchorset = scatac.anchors, reference = scatac.integrated,
    query = scatac.query, new.reduction.name = "ref.pca")
scatac.query <- ProjectUMAP(query = scatac.query, query.reduction = "ref.pca", reference = scatac.integrated,
    reference.reduction = "pca", reduction.model = "umap")


In [ ]:
p1 <- DimPlot(scatac.integrated, reduction = "umap", group.by = "celltype", label = TRUE, label.size = 3,
    repel = TRUE) + NoLegend() + ggtitle("Reference annotations")
p2 <- DimPlot(scatac.query, reduction = "ref.umap", group.by = "predicted.celltype", label = TRUE,
    label.size = 3, repel = TRUE) + NoLegend() + ggtitle("Query transferred labels")
p1 + p2